In [ ]:
! pip install GEOparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 278 kB 17.9 MB/s 
  Created wheel for GEOparse: filename=GEOparse-2.0.3-py3-none-any.whl size=29065 sha256=dc93b21959c5938813e0795318777962b17458c66cd76b08717bf8764809ca2b
  Stored in directory: /root/.cache/pip/wheels/4d/15/e8/fbf3b47444215d9728c20d7b35436b50086aa67c2ad6dcedad
Successfully built GEOparse


In [ ]:
#Import all necessary libraries
import GEOparse
import pandas as pd
import pylab as pl
import seaborn as sns
import numpy as np
import networkx as nx
import scipy
import json
import itertools

In [ ]:
#GSE38837
#Download GEO data
gse = GEOparse.get_GEO(geo="GSE38837")

13-Sep-2022 08:32:56 DEBUG utils - Directory ./ already exists. Skipping.
DEBUG:GEOparse:Directory ./ already exists. Skipping.
13-Sep-2022 08:32:56 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE38nnn/GSE38837/soft/GSE38837_family.soft.gz to ./GSE38837_family.soft.gz
INFO:GEOparse:Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE38nnn/GSE38837/soft/GSE38837_family.soft.gz to ./GSE38837_family.soft.gz
100%|██████████| 11.8M/11.8M [00:00<00:00, 50.1MB/s]
13-Sep-2022 08:32:57 DEBUG downloader - Size validation passed
DEBUG:GEOparse:Size validation passed
13-Sep-2022 08:32:57 DEBUG downloader - Moving /tmp/tmp65p7j_0_ to /content/GSE38837_family.soft.gz
DEBUG:GEOparse:Moving /tmp/tmp65p7j_0_ to /content/GSE38837_family.soft.gz
13-Sep-2022 08:32:57 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE38nnn/GSE38837/soft/GSE38837_family.soft.gz
DEBUG:GEOparse:Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE38nnn/GSE38

In [ ]:
def filter_genes(gse_obj, cut_off, log2, condition_list):
  if log2:
    pivoted_samples = gse_obj.pivot_samples('VALUE')[condition_list]
    return pivoted_samples
  else:
    pivoted_samples = np.log2(gse_obj.pivot_samples('VALUE')[condition_list])
  pivoted_samples_average = pivoted_samples.median(axis=1)
  print("Number of probes before filtering: ", len(pivoted_samples_average))
  expression_threshold = pivoted_samples_average.quantile(cut_off)
  expressed_probes = pivoted_samples_average[pivoted_samples_average >= expression_threshold]
  samples = np.log2(gse.pivot_samples("VALUE").loc[expressed_probes.keys()])
  samples = samples[condition_list]
  print("Number of probes after filtering: ", len(samples))
  return samples

In [ ]:
def get_ttest(control_samples, post_samples):
  ttest_result = scipy.stats.ttest_ind(control_samples, post_samples, axis=1)
  ttest = pd.DataFrame({"stat": ttest_result[0], "pvalue": ttest_result[1]}, index=control_samples.index)

  return ttest

In [ ]:
sample_ids = sorted([key for key in gse.gsms.keys()])
sample_ids

['GSM950616', 'GSM950617', 'GSM950618', 'GSM950619', 'GSM950620', 'GSM950621']

In [ ]:
w_type=sample_ids[0:3]
CA1=sample_ids[3:7]

In [ ]:
w_type

['GSM950616', 'GSM950617', 'GSM950618']

In [ ]:
CA1

['GSM950619', 'GSM950620', 'GSM950621']

In [ ]:
#retina_samples=filter_genes(gse,0.25,True,retina)
w_type_samples=filter_genes(gse,0.25,True,w_type)
CA1_samples=filter_genes(gse,0.25,True,CA1)

In [ ]:
ttest_df = get_ttest(CA1_samples, w_type_samples)

In [ ]:
ttest_df

,stat,pvalue
ID_REF,,
1415670_at,-0.492742,0.648018
1415671_at,-0.698001,0.523624
1415672_at,-1.932359,0.125475
1415673_at,-1.108931,0.329640
1415674_a_at,-0.003426,0.997430
...,...,...
AFFX-r2-Ec-bioC-5_at,-0.018135,0.986399
AFFX-r2-Ec-bioD-3_at,-1.658746,0.172506
AFFX-r2-Ec-bioD-5_at,-1.831805,0.140932


In [ ]:
from statsmodels.stats import multitest


In [ ]:
def get_FDR(ttest_df):
  corrected_pvalue = multitest.multipletests(pvals=ttest_df['pvalue'], method='bonferroni', alpha=0.05)
  print(corrected_pvalue)
  FDR = pd.DataFrame({'Rejected': corrected_pvalue[0], 'FDR': corrected_pvalue[1]}, index= ttest_df.index)

  return FDR

In [ ]:
FDR = get_FDR(ttest_df)

(array([False, False, False, ..., False, False, False]), array([1., 1., 1., ..., 1., 1., 1.]), 1.1372977366530534e-06, 1.108622868672535e-06)


In [ ]:
selected = FDR.loc[FDR['FDR'] < 0.1]
selected

,Rejected,FDR
ID_REF,,
1419044_at,True,0.013166
1424852_at,True,0.009457
1435554_at,True,0.032542
1440053_at,False,0.073954
1455937_at,True,0.002923
1460591_at,False,0.057244


In [ ]:
def get_selected_df(gse_obj, selected_FDR):
  pivoted_samples = gse_obj.pivot_samples('VALUE').loc[selected_FDR.index]
  return pivoted_samples

In [ ]:
selected_df = get_selected_df(gse, selected)
selected_df

name,GSM950616,GSM950617,GSM950618,GSM950619,GSM950620,GSM950621
ID_REF,,,,,,
1419044_at,5.529670,5.570937,5.548845,4.210106,4.180657,4.153062
1424852_at,9.848898,9.906252,9.860385,7.643208,7.715002,7.716190
1435554_at,6.048134,6.067125,6.036805,5.520360,5.513931,5.504152
1440053_at,4.275223,4.264887,4.239616,2.526371,2.442197,2.573967
1455937_at,4.744446,4.780196,4.789321,2.304383,2.323914,2.252748
1460591_at,4.456024,4.504973,4.597879,2.555736,2.527076,2.545011


In [ ]:
selected_df = np.log2(selected_df)
selected_df

name,GSM950616,GSM950617,GSM950618,GSM950619,GSM950620,GSM950621
ID_REF,,,,,,
1419044_at,2.467193,2.477920,2.472188,2.073857,2.063730,2.054175
1424852_at,3.299962,3.308339,3.301644,2.934178,2.947666,2.947889
1435554_at,2.596490,2.601013,2.593785,2.464762,2.463081,2.460520
1440053_at,2.096000,2.092508,2.083933,1.337066,1.288180,1.363994
1455937_at,2.246240,2.257070,2.259821,1.204380,1.216556,1.171686
1460591_at,2.155757,2.171519,2.200968,1.353739,1.337469,1.347672


In [ ]:
def get_lfc(control_samples, post_samples):
  control_mean = control_samples.mean(axis=1)
  post_mean = post_samples.mean(axis=1)
  LFCs = pd.DataFrame({"LFC": (post_mean - control_mean).to_dict()})
  return LFCs

In [ ]:
selected_df[CA1]

name,GSM950619,GSM950620,GSM950621
ID_REF,,,
1419044_at,2.073857,2.063730,2.054175
1424852_at,2.934178,2.947666,2.947889
1435554_at,2.464762,2.463081,2.460520
1440053_at,1.337066,1.288180,1.363994
1455937_at,1.204380,1.216556,1.171686
1460591_at,1.353739,1.337469,1.347672


In [ ]:
selected_df[w_type]

name,GSM950616,GSM950617,GSM950618
ID_REF,,,
1419044_at,2.467193,2.477920,2.472188
1424852_at,3.299962,3.308339,3.301644
1435554_at,2.596490,2.601013,2.593785
1440053_at,2.096000,2.092508,2.083933
1455937_at,2.246240,2.257070,2.259821
1460591_at,2.155757,2.171519,2.200968


In [ ]:
#huvec_samples = selected_df[huvec]
CA1_sample=selected_df[CA1]
w_type_sample=selected_df[w_type]
#LFCs = get_lfc(huvec_samples, choroid_samples)
LFCs = get_lfc(CA1_sample,w_type_sample)
LFCs

,LFC
1419044_at,0.408513
1424852_at,0.360071
1435554_at,0.134308
1440053_at,0.761067
1455937_at,1.056836
1460591_at,0.829788


In [ ]:
def get_annotation(gene_dataframe, data_flatform, leftkey):
  gene_annotated = gene_dataframe.reset_index().merge(gse.gpls[data_flatform].table[["ID", "ENTREZ_GENE_ID", "Gene Symbol"]],
                                left_on=leftkey, right_on="ID").set_index(leftkey)
  del gene_annotated["ID"]
  # remove probes without ENTREZ
  gene_annotated = gene_annotated.dropna(subset=["ENTREZ_GENE_ID"])
  # remove probes with more than one gene assigned
  gene_annotated['ENTREZ_GENE_ID'] = pd.to_numeric(gene_annotated['ENTREZ_GENE_ID'], errors="coerce")
  gene_annotated.dropna(how="any", inplace=True)
  gene_annotated['ENTREZ_GENE_ID'] = gene_annotated.ENTREZ_GENE_ID.astype('int').astype('str')
  # for each gene average LFC over probes
  gene_annotated = gene_annotated.groupby("Gene Symbol").median()

  return gene_annotated

In [ ]:
#LFCs_annotated = get_annotation(LFCs, 'GPL570', 'index')
#GPL1261
LFCs_annotated = get_annotation(LFCs, 'GPL1261', 'index')

In [ ]:
LFCs_annotated

,LFC
Gene Symbol,
Cntnap4,0.408513
Esr1,0.829788
Lrrtm4,1.056836
Mef2c,0.360071
Tmcc3,0.134308
